<a href="https://colab.research.google.com/github/boskein/Awesome-Onion-Links/blob/master/Notebooks/Google%20AI/Gemini/Tools/gemini_file_search_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Tutorial Completo: Gemini File Search - Sistema RAG Profesional

## 📚 Índice

1. [Introducción y Setup](#1-introducción)
2. [Crear File Search Store](#2-crear-file-search-store)
3. [Subir y Indexar Documentos](#3-subir-documentos)
4. [Realizar Búsquedas Básicas](#4-búsquedas-básicas)
5. [Obtener Citaciones](#5-citaciones)
6. [Metadata Filtering Avanzado](#6-metadata-filtering)
7. [Gestión de Stores](#7-gestión-stores)
8. [Chunking Personalizado](#8-chunking-personalizado)
9. [Mejores Prácticas](#9-mejores-prácticas)

---

## ℹ️ Acerca de este Tutorial

**Nivel:** Intermedio  
**Duración estimada:** 30-40 minutos  
**Requisitos previos:**
- Python 3.9+
- Conocimientos básicos de Python
- API Key de Google AI Studio

**¿Qué aprenderás?**
- Implementar un sistema RAG completo sin gestionar infraestructura
- Indexar documentos de múltiples formatos
- Realizar búsquedas semánticas avanzadas
- Filtrar resultados con metadata
- Obtener citaciones automáticas

---

## 1. 🔧 Introducción y Setup

### ¿Qué es File Search?

File Search es un sistema RAG (Retrieval Augmented Generation) completamente gestionado por Google que te permite:

- ✅ **Subir documentos** de múltiples formatos (PDF, DOCX, TXT, JSON, código, etc.)
- ✅ **Indexación automática** con chunking inteligente y embeddings de última generación
- ✅ **Búsqueda semántica** que entiende significado, no solo palabras clave
- ✅ **Citaciones automáticas** para verificabilidad
- ✅ **Sin infraestructura** - no necesitas Pinecone, Chroma, ni gestionar bases vectoriales

### 💰 Precios

- **Storage:** GRATIS ✨
- **Embeddings en indexación:** $0.15 por 1M tokens (una sola vez al subir)
- **Embeddings en búsqueda:** GRATIS
- **Tokens recuperados:** Se cobran como context tokens del modelo

### 📦 Instalación

In [1]:
# Instalar la librería de Google Generative AI
!pip install google-genai -q

### 🔑 Configurar API Key

**Opción 1: Variable de entorno (recomendado para producción)**

In [4]:
import os
from getpass import getpass

# Si no tienes la API key en variables de entorno, ingrésala aquí
if 'GOOGLE_API_KEY' not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass('Ingresa tu API Key de Google AI Studio: ')

API_KEY = os.environ['GOOGLE_API_KEY']
print("✅ API Key configurada")

✅ API Key configurada


**Opción 2: Directamente en código (solo para desarrollo)**

In [ ]:
# ⚠️ NO USES ESTO EN PRODUCCIÓN - Solo para testing local
# API_KEY = 'TU_API_KEY_AQUI'

**Opción 3: Secretos (Solo para google Colab)**

In [3]:
from google.colab import userdata
API_KEY = userdata.get('GEMINI_API_KEY')

SecretNotFoundError: Secret GEMINI_API_KEY does not exist.

### 📚 Imports necesarios

In [7]:
from google import genai
from google.genai import types
import time
from typing import List, Dict, Optional
import json

# Inicializar el cliente de Gemini
client = genai.Client(api_key=API_KEY)

print("✅ Cliente inicializado correctamente")

✅ Cliente inicializado correctamente


---

## 2. 📦 Crear File Search Store

Un **File Search Store** es un contenedor para tus documentos indexados. Piénsalo como una base de datos especializada.

### Mejores prácticas:
- 🗂️ Crea stores separados por tipo de contenido (ej: docs públicas vs internas)
- 📝 Usa `display_name` descriptivo para identificar fácilmente
- 🎯 Mantén cada store bajo 20GB para latencias óptimas

In [8]:
def create_file_search_store(display_name: str) -> genai.types.FileSearchStore:
    """
    Crea un nuevo File Search Store.

    Args:
        display_name: Nombre descriptivo para identificar el store

    Returns:
        FileSearchStore object con información del store creado
    """
    store = client.file_search_stores.create(
        config={'display_name': display_name}
    )

    print(f"✅ Store creado exitosamente")
    print(f"   • Nombre: {store.name}")
    print(f"   • Display Name: {display_name}")

    return store

In [9]:

# Crear nuestro primer store
store = create_file_search_store("reporte_1")

✅ Store creado exitosamente
   • Nombre: fileSearchStores/reporte1-eg6nnoq7rwb7
   • Display Name: reporte_1


In [10]:

# Crear nuestro primer store
store_script = create_file_search_store("reporte_2")

✅ Store creado exitosamente
   • Nombre: fileSearchStores/reporte2-fxyfq9pnwsow
   • Display Name: reporte_2


---

## 3. 📄 Subir y Indexar Documentos

### Formatos soportados:
- 📑 Documentos: PDF, DOCX, TXT, MD, RTF
- 💻 Código: Python, JavaScript, Java, C++, Go, Rust, y más
- 📊 Datos: JSON, CSV, XML
- 🌐 Web: HTML

### Método 1: Upload directo (recomendado)

In [11]:
def upload_file_to_store(
    file_path: str,
    store_name: str,
    display_name: Optional[str] = None,
    custom_metadata: Optional[List[Dict]] = None
) -> bool:
    """
    Sube e indexa un archivo directamente en un File Search Store.

    Args:
        file_path: Ruta local al archivo
        store_name: Nombre del store (ej: 'fileSearchStores/abc123')
        display_name: Nombre para identificar el archivo (opcional)
        custom_metadata: Lista de metadatos personalizados (opcional)

    Returns:
        bool: True si la indexación fue exitosa
    """
    try:
        config = {}
        if display_name:
            config['display_name'] = display_name
        if custom_metadata:
            config['custom_metadata'] = custom_metadata

        print(f"📤 Subiendo: {file_path}...")

        # Iniciar la operación de upload
        operation = client.file_search_stores.upload_to_file_search_store(
            file=file_path,
            file_search_store_name=store_name,
            config=config if config else None
        )

        # Esperar a que la indexación complete
        print(f"⏳ Indexando (esto puede tomar algunos segundos)...")
        while not operation.done:
            time.sleep(5)
            operation = client.operations.get(operation)

        print(f"✅ Archivo indexado: {display_name or file_path}\n")
        return True

    except Exception as e:
        print(f"❌ Error indexando {file_path}: {str(e)}\n")
        return False

### Ejemplo: Crear un documento de prueba y subirlo

In [15]:
# Subir el documento
upload_file_to_store(
    file_path='/content/WEF_Future_of_Jobs_Report_2025.pdf',
    store_name=store.name,
    display_name='Reporte',
    custom_metadata=[
        {'key': 'tipo', 'string_value': 'guía'},
        {'key': 'idioma', 'string_value': 'english'},
        {'key': 'version', 'numeric_value': 1}
    ]
)

📤 Subiendo: /content/WEF_Future_of_Jobs_Report_2025.pdf...
⏳ Indexando (esto puede tomar algunos segundos)...
✅ Archivo indexado: Reporte



True

In [16]:
# Subir el documento
upload_file_to_store(
    file_path='/content/informe-colombia.pdf',
    store_name=store_script.name,
    display_name='Informe Colombia',
    custom_metadata=[
        {'key': 'tipo', 'string_value': 'script'},
        {'key': 'version', 'numeric_value': 2}
    ]
)

📤 Subiendo: /content/informe-colombia.pdf...
⏳ Indexando (esto puede tomar algunos segundos)...
✅ Archivo indexado: Informe Colombia



True

### Subir múltiples archivos de forma eficiente

In [17]:
def batch_upload_files(
    file_paths: List[str],
    store_name: str,
    metadata_generator: Optional[callable] = None
) -> Dict[str, bool]:
    """
    Sube múltiples archivos de forma secuencial.

    Args:
        file_paths: Lista de rutas a archivos
        store_name: Nombre del store
        metadata_generator: Función que genera metadata para cada archivo

    Returns:
        Dict con el resultado de cada upload (filename: success_bool)
    """
    results = {}

    print(f"📦 Iniciando batch upload de {len(file_paths)} archivos...\n")

    for file_path in file_paths:
        metadata = metadata_generator(file_path) if metadata_generator else None

        success = upload_file_to_store(
            file_path=file_path,
            store_name=store_name,
            display_name=file_path,
            custom_metadata=metadata
        )

        results[file_path] = success

    # Resumen
    successful = sum(1 for v in results.values() if v)
    print(f"\n📊 Resumen: {successful}/{len(file_paths)} archivos indexados exitosamente")

    return results

# Ejemplo: crear y subir varios documentos
# (Descomenta si quieres probar con múltiples archivos)

# archivos = ['doc1.txt', 'doc2.txt', 'doc3.txt']
# results = batch_upload_files(archivos, store.name)

---

## 4. 🔍 Realizar Búsquedas Básicas

Ahora que tenemos documentos indexados, podemos hacer consultas en lenguaje natural.

### Cómo funciona:
1. Tu consulta se convierte en un embedding
2. Se buscan los chunks más similares semánticamente
3. Los chunks relevantes se inyectan en el contexto del modelo
4. El modelo genera una respuesta fundamentada

In [24]:
def search_file_store(
    query: str,
    store_names: List[str],
    model: str = 'gemini-2.5-flash',
    metadata_filter: Optional[str] = None
) -> genai.types.GenerateContentResponse:
    """
    Realiza una búsqueda en uno o más File Search Stores.

    Args:
        query: Pregunta o consulta en lenguaje natural
        store_names: Lista de stores donde buscar
        model: Modelo de Gemini a usar (2.5-flash o 2.5-pro)
        metadata_filter: Filtro opcional de metadata

    Returns:
        Respuesta del modelo con el contenido generado
    """
    # Configurar la herramienta de File Search
    file_search_config = types.FileSearch(
        file_search_store_names=store_names
    )

    if metadata_filter:
        file_search_config.metadata_filter = metadata_filter

    # Hacer la consulta
    response = client.models.generate_content(
        model=model,
        contents=query,
        config=types.GenerateContentConfig(
            tools=[
                types.Tool(
                    file_search=file_search_config
                )
            ]
        )
    )

    return response

[texto del enlace](https://)### Ejemplo: Consultas básicas

In [28]:
# Consulta 1: Precio
print("❓ Pregunta: ¿Cuál es el costo de usar File Search?\n")

response = search_file_store(
    query="EHS Maturity curve percentage in stage 2",
    store_names=[store_script.name]
)

print(f"💬 Respuesta:\n{response.text}\n")
print("-" * 80)

❓ Pregunta: ¿Cuál es el costo de usar File Search?

💬 Respuesta:
According to the EHS Maturity Curve, 67.1% of organizations are in Stage 2, which is categorized as "Operational." In this stage, organizations focus on workforce engagement and visibility, utilizing point solutions for core areas, though some processes may still be paper and spreadsheet-based. This stage also involves limited digitization.

--------------------------------------------------------------------------------


In [16]:
# Consulta 2: Características
print("❓ Pregunta: ¿Qué formatos de archivo soporta?\n")

response = search_file_store(
    query="¿Valor global EHS SW 2024-25?",
    store_names=[store.name]
)

print(f"💬 Respuesta:\n{response.text}\n")
print("-" * 80)

❓ Pregunta: ¿Qué formatos de archivo soporta?

💬 Respuesta:
Lo siento, no pude encontrar el valor global del software EHS para 2024-2025 en los documentos proporcionados. Los informes se centran en tendencias económicas, perspectivas laborales, habilidades y otros aspectos, pero no en el tamaño del mercado del software EHS.

--------------------------------------------------------------------------------


In [66]:
# Consulta 3: Casos de uso
print("❓ Pregunta: ¿Para qué casos de uso se recomienda?\n")

response = search_file_store(
    query="El software de Environment, Health & Safety (EHS) pasará de ",
    store_names=[store.name]
)

print(f"💬 Respuesta:\n{response.text}\n")
print("-" * 80)

❓ Pregunta: ¿Para qué casos de uso se recomienda?

💬 Respuesta:
El software de Environment, Health & Safety (EHS) está evolucionando para incorporar nuevas tendencias y prioridades. Aunque la información proporcionada no detalla una transición específica "de X a Y" para el software EHS, se pueden inferir varias direcciones clave basadas en las tendencias generales del mercado laboral y las estrategias empresariales:

*   **De la gestión reactiva a la proactiva y basada en datos:** Se espera una creciente demanda de habilidades en IA y big data, así como un enfoque en el pensamiento analítico. Esto sugiere que el software EHS se moverá hacia soluciones que utilicen grandes volúmenes de datos para análisis predictivos y una gestión más proactiva de la seguridad y el medio ambiente, en lugar de solo registrar incidentes pasados.
*   **Mayor integración con la sostenibilidad y la gestión ambiental:** Las empresas están priorizando la administración ambiental y la reducción de emisiones de 

In [ ]:
# Consulta 4: Casos de uso
print("❓ Pregunta: Cual es el nombre del ganso?\n")

response = search_file_store(
    query="Cual es el nombre del ganso?",
    store_names=[store_script.name]
)

print(f"💬 Respuesta:\n{response.text}\n")
print("-" * 80)

❓ Pregunta: Cual es el nombre del ganso?

💬 Respuesta:
El nombre del ganso es Brightbill. Inicialmente, Roz lo nombró "Gosling zero-zero-zero-one", pero más tarde le dio el nombre de Brightbill.

--------------------------------------------------------------------------------


In [ ]:
# Consulta 4: Casos de uso
print("❓ Pregunta: Quien es Fink, responde en español?\n")

response = search_file_store(
    query="Quien es Fink, responde en español?",
    store_names=[store_script.name]
)

print(f"💬 Respuesta:\n{response.text}\n")
print("-" * 80)

❓ Pregunta: Quien es Fink, responde en español?

💬 Respuesta:
Fink es un personaje de la película "Wild Robot". Se presenta como un zorro, un depredador astuto y resbaladizo, y un experto local en gansos.

Algunas características y acciones clave de Fink incluyen:
*   **Naturaleza depredadora y engañosa:** Inicialmente, Fink intenta arrebatar un gansito a Rozzum 7134, explicando que es su naturaleza de zorro hacer "cosas de zorro". También se muestra manipulador, preguntando si Rozzum 7134 cree todo lo que escucha después de decir que no se comería al gansito, para luego retractarse de inmediato.
*   **Ayuda al gansito:** A pesar de su naturaleza, Fink ayuda a Rozzum 7134 a cuidar al gansito, enseñándole a comer, nadar y volar. Sin embargo, sus métodos a menudo son egoístas, como cuando se come las vieiras y la miel destinadas al gansito.
*   **Comportamiento y personalidad:** Fink es glotón, se le ve comiendo frecuentemente y defendiendo su comida, incluso robando salmón de Thorn. Tam

---

## 5. 📚 Obtener Citaciones

Las citaciones te permiten verificar de dónde viene cada parte de la respuesta.

### Por qué son importantes:
- ✅ Verificabilidad y confianza
- ✅ Cumplimiento y auditoría
- ✅ Debugging de respuestas incorrectas
- ✅ Transparencia en aplicaciones críticas

In [ ]:
def search_with_citations(
    query: str,
    store_names: List[str],
    show_full_chunks: bool = False
) -> None:
    """
    Realiza una búsqueda y muestra las citaciones de forma legible.

    Args:
        query: Pregunta en lenguaje natural
        store_names: Lista de stores donde buscar
        show_full_chunks: Si True, muestra el texto completo de cada chunk
    """
    print(f"❓ Pregunta: {query}\n")

    # Realizar búsqueda
    response = search_file_store(query, store_names)

    # Mostrar respuesta
    print(f"💬 Respuesta:\n{response.text}\n")
    print("=" * 80)

    # Obtener citaciones
    grounding = response.candidates[0].grounding_metadata

    if not grounding or not grounding.grounding_chunks:
        print("ℹ️  No hay citaciones disponibles para esta respuesta")
        return

    print(f"\n📖 CITACIONES ({len(grounding.grounding_chunks)} fuentes):\n")

    for i, chunk in enumerate(grounding.grounding_chunks, 1):
        context = chunk.retrieved_context

        print(f"[{i}] 📄 Fuente: {context.title}")

        if show_full_chunks:
            print(f"    Contenido:\n{context.text}\n")
        else:
            # Mostrar solo un extracto
            preview = context.text[:200] + "..." if len(context.text) > 200 else context.text
            print(f"    Extracto: {preview}\n")

In [ ]:
# Ejemplo: Búsqueda con citaciones
search_with_citations(
    query="¿Cuáles son los límites de almacenamiento según el tier del usuario?",
    store_names=[store.name],
    show_full_chunks=False  # Cambia a True para ver el texto completo
)

❓ Pregunta: ¿Cuáles son los límites de almacenamiento según el tier del usuario?

💬 Respuesta:
Los límites de almacenamiento total para los almacenes de File Search de un proyecto varían según el nivel de usuario:

*   **Gratuito:** 1 GB
*   **Nivel 1:** 10 GB
*   **Nivel 2:** 100 GB
*   **Nivel 3:** 1 TB

Además, el tamaño máximo por archivo o por documento está limitado a 100 MB. Se recomienda que el tamaño de cada almacén de File Search sea inferior a 20 GB para asegurar latencias óptimas de recuperación.

Es importante tener en cuenta que el límite de tamaño del almacén de File Search se calcula en el backend, basándose en el tamaño de la entrada más los embeddings generados y almacenados con ella, lo que suele ser aproximadamente 3 veces el tamaño de los datos de entrada. El almacenamiento en sí no tiene costo.


📖 CITACIONES (3 fuentes):

[1] 📄 Fuente: Guía Completa de File Search
    Extracto: text/x-dsrc

text/x-emacs-lisp

text/x-erlang

text/x-gff3

text/x-go

• text/x-haskel

---

## 6. 🏷️ Metadata Filtering Avanzado

El metadata filtering te permite hacer búsquedas ultra-precisas en subconjuntos de tus documentos.

### Sintaxis soportada:
- `key = 'value'` - Igualdad exacta
- `key != 'value'` - Diferente de
- `key > 100` - Mayor que (numérico)
- `key >= 100` - Mayor o igual
- `key < 100` - Menor que
- `key <= 100` - Menor o igual
- `key IN ['val1', 'val2']` - Uno de varios valores
- Combinar con `AND` y `OR`

### Crear documentos con metadata rica

In [ ]:
# Crear varios documentos con diferentes metadata
docs_with_metadata = [
    {
        'filename': 'pricing_2025.txt',
        'content': 'Precios 2025: File Search cobra $0.15 por millón de tokens en indexación. Storage es gratuito.',
        'metadata': [
            {'key': 'anio', 'numeric_value': 2025},
            {'key': 'categoria', 'string_value': 'pricing'},
            {'key': 'departamento', 'string_value': 'finanzas'}
        ]
    },
    {
        'filename': 'features_2024.txt',
        'content': 'Características 2024: Soporte para PDF, DOCX, TXT, código fuente. Búsqueda semántica avanzada.',
        'metadata': [
            {'key': 'anio', 'numeric_value': 2024},
            {'key': 'categoria', 'string_value': 'features'},
            {'key': 'departamento', 'string_value': 'producto'}
        ]
    },
    {
        'filename': 'pricing_2023.txt',
        'content': 'Precios 2023: En la versión anterior era de 0.5$ usd, el pricing era diferente. Versión legacy.',
        'metadata': [
            {'key': 'anio', 'numeric_value': 2023},
            {'key': 'categoria', 'string_value': 'pricing'},
            {'key': 'departamento', 'string_value': 'finanzas'},
            {'key': 'legacy', 'string_value': 'true'}
        ]
    }
]

# Subir los documentos
for doc in docs_with_metadata:
    # Crear archivo temporal
    with open(doc['filename'], 'w', encoding='utf-8') as f:
        f.write(doc['content'])

    # Subir con metadata
    upload_file_to_store(
        file_path=doc['filename'],
        store_name=store.name,
        display_name=doc['filename'],
        custom_metadata=doc['metadata']
    )

print("✅ Documentos con metadata subidos")

📤 Subiendo: pricing_2025.txt...
⏳ Indexando (esto puede tomar algunos segundos)...
✅ Archivo indexado: pricing_2025.txt

📤 Subiendo: features_2024.txt...
⏳ Indexando (esto puede tomar algunos segundos)...
✅ Archivo indexado: features_2024.txt

📤 Subiendo: pricing_2023.txt...
⏳ Indexando (esto puede tomar algunos segundos)...
✅ Archivo indexado: pricing_2023.txt

✅ Documentos con metadata subidos


### Ejemplos de filtrado

In [ ]:
# Filtro 1: Solo documentos de 2024
print("🔍 Filtro: anio = 2025\n")

response = search_file_store(
    query="¿Cuál es el precio actual?",
    store_names=[store.name],
    metadata_filter="anio=2025"
)

print(f"Respuesta: {response.text}\n")
print("-" * 80)

🔍 Filtro: anio = 2025

Respuesta: El precio actual para la indexación con File Search es de $0.15 por millón de tokens. El almacenamiento es gratuito.

--------------------------------------------------------------------------------


In [ ]:
# Filtro 2: Rango de años
print("🔍 Filtro: anio >= 2024\n")

response = search_file_store(
    query="Dame la información de Características",
    store_names=[store.name],
    metadata_filter="anio>=2024"
)

print(f"Respuesta: {response.text}\n")
print("-" * 80)

🔍 Filtro: anio >= 2024

Respuesta: Las características disponibles para el año 2024 incluyen soporte para diferentes formatos de archivo como PDF, DOCX, TXT y código fuente. También se destaca la funcionalidad de búsqueda semántica avanzada.

--------------------------------------------------------------------------------


In [ ]:
# Filtro 3: Múltiples categorías con OR
print("🔍 Filtro: departamento finanzas\n")

response = search_file_store(
    query="Hay precios para 2023?",
    store_names=[store.name],
    metadata_filter="departamento=finanzas"
)

print(f"Respuesta: {response.text}\n")
print("-" * 80)

🔍 Filtro: departamento finanzas

Respuesta: Sí, hay información sobre precios para 2023. En la versión anterior, el precio era de 0.5 USD, y se refiere a una versión legacy.

--------------------------------------------------------------------------------


---

## 7. 🗂️ Gestión de Stores

Funciones para listar, obtener información, y eliminar stores.

In [51]:
def list_all_stores() -> List[genai.types.FileSearchStore]:
    """
    Lista todos los File Search Stores del usuario.

    Returns:
        Lista de FileSearchStore objects
    """
    stores = list(client.file_search_stores.list())

    print(f"📚 Tienes {len(stores)} store(s):\n")

    for i, store in enumerate(stores, 1):
        print(f"{i}. {store.display_name or 'Sin nombre'}")
        print(f"   • ID: {store.name}")
        print(f"   • Creado: {store.create_time}\n")

    return stores

# Listar stores
my_stores = list_all_stores()

📚 Tienes 4 store(s):

1. Tutorial Gemini File Search - Documentación
   • ID: fileSearchStores/tutorial-gemini-file-search-iklce8ioifrj
   • Creado: 2025-11-26 14:22:19.165785+00:00

2. Script peliculas
   • ID: fileSearchStores/script-peliculas-ajfi4b7f9nli
   • Creado: 2025-11-26 14:23:56.660188+00:00

3. Script peliculas
   • ID: fileSearchStores/script-peliculas-4hkv65d8r8vd
   • Creado: 2025-11-26 14:50:45.579962+00:00

4. Tutorial Gemini File Search - Documentación
   • ID: fileSearchStores/tutorial-gemini-file-search-i8e1b0k6d5t7
   • Creado: 2025-11-26 14:50:46.345975+00:00



In [52]:
def get_store_info(store_name: str) -> None:
    """
    Obtiene información detallada de un store específico.

    Args:
        store_name: Nombre completo del store
    """
    try:
        store = client.file_search_stores.get(name=store_name)

        print(f"📊 Información del Store:\n")
        print(f"Nombre: {store.display_name or 'Sin nombre'}")
        print(f"ID: {store.name}")
        print(f"Creado: {store.create_time}")
        print(f"Actualizado: {store.update_time}")

    except Exception as e:
        print(f"❌ Error obteniendo info del store: {str(e)}")

# Ver info del store actual
get_store_info(store.name)

📊 Información del Store:

Nombre: Tutorial Gemini File Search - Documentación
ID: fileSearchStores/tutorial-gemini-file-search-i8e1b0k6d5t7
Creado: 2025-11-26 14:50:46.345975+00:00
Actualizado: 2025-11-26 14:50:46.345975+00:00


In [ ]:
def delete_store(store_name: str, confirm: bool = False) -> bool:
    """
    Elimina un File Search Store.

    ⚠️ ADVERTENCIA: Esta acción es IRREVERSIBLE.
    Se perderán todos los documentos indexados.

    Args:
        store_name: Nombre del store a eliminar
        confirm: Debe ser True para confirmar la eliminación

    Returns:
        bool: True si se eliminó exitosamente
    """
    if not confirm:
        print("⚠️  ADVERTENCIA: Debes pasar confirm=True para eliminar el store")
        print("   Esta acción es IRREVERSIBLE y eliminará todos los documentos indexados.")
        return False

    try:
        client.file_search_stores.delete(name=store_name, config={'force': True})
        print(f"✅ Store eliminado: {store_name}")
        return True

    except Exception as e:
        print(f"❌ Error eliminando store: {str(e)}")
        return False

# Ejemplo (comentado por seguridad)
delete_store('fileSearchStores/mi-documentacin-tcnica-0uml3wlpg780', confirm=True)

✅ Store eliminado: fileSearchStores/mi-documentacin-tcnica-0uml3wlpg780


True

---

## 8. ⚙️ Chunking Personalizado

Por defecto, File Search usa chunking automático inteligente. Pero puedes personalizarlo si necesitas control específico.

In [ ]:
def upload_with_custom_chunking(
    file_path: str,
    store_name: str,
    max_tokens_per_chunk: int = 500,
    max_overlap_tokens: int = 50
) -> bool:
    """
    Sube un archivo con configuración personalizada de chunking.

    Args:
        file_path: Ruta al archivo
        store_name: Nombre del store
        max_tokens_per_chunk: Máximo de tokens por chunk (default: 500)
        max_overlap_tokens: Tokens de overlap entre chunks (default: 50)

    Returns:
        bool: True si fue exitoso
    """
    try:
        print(f"📤 Subiendo con chunking personalizado:")
        print(f"   • Max tokens por chunk: {max_tokens_per_chunk}")
        print(f"   • Overlap: {max_overlap_tokens} tokens\n")

        operation = client.file_search_stores.upload_to_file_search_store(
            file=file_path,
            file_search_store_name=store_name,
            config={
                'display_name': file_path,
                'chunking_config': {
                    'white_space_config': {
                        'max_tokens_per_chunk': max_tokens_per_chunk,
                        'max_overlap_tokens': max_overlap_tokens
                    }
                }
            }
        )

        # Esperar indexación
        while not operation.done:
            time.sleep(5)
            operation = client.operations.get(operation)

        print(f"✅ Archivo indexado con chunking personalizado\n")
        return True

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return False

In [ ]:
# Crear documento de prueba con texto largo
long_content = """
Chunking en RAG: Guía Completa

El chunking es el proceso de dividir documentos largos en segmentos más pequeños.
Esto es crucial porque los modelos tienen límites de contexto y necesitamos
recuperar solo la información relevante.

Estrategias de Chunking:

1. Fixed-size chunking: Dividir por número fijo de tokens o caracteres.
   Ventajas: Simple de implementar.
   Desventajas: Puede cortar contexto importante.

2. Semantic chunking: Dividir por significado semántico.
   Ventajas: Mantiene contexto coherente.
   Desventajas: Más complejo de implementar.

3. Recursive chunking: Dividir jerárquicamente (párrafos, oraciones, etc.).
   Ventajas: Balancea tamaño y coherencia.
   Desventajas: Requiere análisis estructural.

File Search usa una combinación de estas estrategias automáticamente.

El overlap entre chunks es importante para:
- No perder contexto en los bordes
- Mejorar recuperación de información fragmentada
- Aumentar recall en búsquedas

Un buen overlap típico es 10-20% del tamaño del chunk.
""" * 3  # Repetir para hacer el documento más largo

with open('chunking_guide.txt', 'w', encoding='utf-8') as f:
    f.write(long_content)

# Subir con chunking personalizado
upload_with_custom_chunking(
    file_path='chunking_guide.txt',
    store_name=store.name,
    max_tokens_per_chunk=200,  # Chunks más pequeños
    max_overlap_tokens=40      # 20% de overlap
)

📤 Subiendo con chunking personalizado:
   • Max tokens por chunk: 200
   • Overlap: 40 tokens

✅ Archivo indexado con chunking personalizado



True

---

## 9. 💡 Mejores Prácticas

### ✅ DO's (Hazlo)

```python
# ✅ Organizar stores por tipo de contenido
docs_publicas = create_file_search_store("Documentación Pública")
docs_internas = create_file_search_store("Documentación Interna")
training_data = create_file_search_store("Datos de Entrenamiento")

# ✅ Usar metadata rica para filtrado preciso
metadata = [
    {'key': 'departamento', 'string_value': 'ingenieria'},
    {'key': 'fecha', 'numeric_value': 20240101},
    {'key': 'confidencial', 'string_value': 'no'},
    {'key': 'version', 'numeric_value': 2}
]

# ✅ Manejar errores apropiadamente
try:
    response = search_file_store(query, [store.name])
except Exception as e:
    logging.error(f"Error en búsqueda: {e}")
    # Fallback o retry logic

# ✅ Validar citaciones en aplicaciones críticas
if not response.candidates[0].grounding_metadata:
    print("⚠️ Respuesta sin citaciones - validar manualmente")

# ✅ Usar display_names descriptivos
upload_file_to_store(
    file='doc.pdf',
    store_name=store.name,
    display_name='API Documentation v2.3 - Authentication Module'
)

# ✅ Implementar rate limiting en producción
import time
from functools import wraps

def rate_limit(calls_per_minute=10):
    def decorator(func):
        last_calls = []
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()
            last_calls[:] = [t for t in last_calls if now - t < 60]
            
            if len(last_calls) >= calls_per_minute:
                sleep_time = 60 - (now - last_calls[0])
                time.sleep(sleep_time)
            
            last_calls.append(time.time())
            return func(*args, **kwargs)
        
        return wrapper
    return decorator
```

### ❌ DON'Ts (No lo hagas)

```python
# ❌ NO subir archivos sin esperar a que la indexación complete
operation = client.file_search_stores.upload_to_file_search_store(...)
# Falta el while loop para esperar
search_file_store(query, [store.name])  # Puede fallar!

# ❌ NO poner credenciales hardcoded
API_KEY = 'AIzaSyXXXXXXXX'  # Nunca hagas esto!

# ✅ Usa variables de entorno
API_KEY = os.getenv('GOOGLE_API_KEY')

# ❌ NO ignorar los límites de tamaño
# Archivo de 150MB - excede el límite de 100MB
upload_file_to_store('huge_file.pdf', store.name)  # Fallará!

# ✅ Verificar tamaño antes de subir
import os
file_size_mb = os.path.getsize('file.pdf') / (1024 * 1024)
if file_size_mb > 100:
    print("Archivo muy grande - considerar dividir")

# ❌ NO usar modelos no soportados
search_file_store(
    query="test",
    store_names=[store.name],
    model='gemini-1.5-pro'  # No soportado!
)

# ✅ Usar solo modelos soportados
supported_models = ['gemini-2.5-flash', 'gemini-2.5-pro']

# ❌ NO asumir que siempre hay citaciones
sources = response.candidates[0].grounding_metadata.grounding_chunks[0]  # Puede ser None!

# ✅ Verificar primero
grounding = response.candidates[0].grounding_metadata
if grounding and grounding.grounding_chunks:
    sources = grounding.grounding_chunks

# ❌ NO crear un store gigante con todo
mega_store = create_file_search_store("Todo")
# Sube docs públicos, internos, código, etc. todo junto - mala práctica!

# ✅ Separar por contexto y seguridad
public_store = create_file_search_store("Público")
private_store = create_file_search_store("Interno")
```